In [1]:
from azureml.core import Workspace, Experiment

# Configure workspace and experiment
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="titanic-classification")

In [2]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpu-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target: cpu-cluster


In [3]:
import pandas as pd
df = pd.read_csv('titanic/train.csv')
df = df.drop_duplicates()

target_column = "Survived"

In [4]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2)

In [5]:
automl_settings = {
 "experiment_timeout_minutes" : 15,
 "n_cross_validations": 3,
 "primary_metric": 'accuracy',
 "featurization": 'auto',
 "preprocess": True,
 "max_concurrent_iterations": 4
}

In [6]:
from azureml.core import Dataset
from azureml.data.dataset_factory import DataType

# create a TabularDataset from a delimited file behind a public web url and convert column "Survived" to boolean
web_path ='https://dprepdata.blob.core.windows.net/demo/Titanic.csv'
titanic_ds = Dataset.Tabular.from_delimited_files(path=web_path, set_column_types={'Survived': DataType.to_bool()})

# preview the first 3 rows of titanic_ds
titanic_ds.take(3).to_pandas_dataframe()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,False,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,,S
1,2,True,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,True,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,,S


In [7]:
train_dataset, test_dataset = titanic_ds.random_split(percentage=0.1, seed=42)

In [8]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task = 'classification',
 debug_log = 'debug.log',
 compute_target = compute_target,
 training_data = train_dataset,
 label_column_name = target_column,
 **automl_settings)

In [9]:
automl_run = exp.submit(automl_config)
print(automl_run.get_portal_url())

https://ml.azure.com/experiments/titanic-classification/runs/AutoML_19437e5e-9a46-4782-92a5-fadb98a84d62?wsid=/subscriptions/21dc412b-d9eb-42e7-8317-55bc8eb10cf5/resourcegroups/packt-mastering-azure-machine-learning/workspaces/packt


In [10]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [11]:
automl_run.wait_for_completion(show_output=False)

{'runId': 'AutoML_19437e5e-9a46-4782-92a5-fadb98a84d62',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-04-27T08:20:55.770459Z',
 'endTimeUtc': '2020-04-27T08:38:21.138364Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'cpu-cluster',
  'RawAMLSettingsString': "{'name': 'titanic-classification', 'path': None, 'subscription_id': '21dc412b-d9eb-42e7-8317-55bc8eb10cf5', 'resource_group': 'packt-mastering-azure-machine-learning', 'workspace_name': 'packt', 'region': 'westeurope', 'compute_target': 'cpu-cluster', 'spark_service': None, 'azure_service': None, 'iterations': 1000, 'primary_metric': 'accuracy', 'task_type': 'classification', 'data_script': None, 'validation_size': 0.0, 'n_cross_validations': 3, 'y_min': None, 'y_max': None, 'num_classes': None, 'featurization': 'aut

In [12]:
best_run, fitted_model = automl_run.get_output()

In [13]:
from sklearn.metrics import accuracy_score

df_test = test_dataset.to_pandas_dataframe()

y_test = df_test[target_column]
X_test = df_test.drop(target_column, axis=1)
y_pred = fitted_model.predict(X_test)

accuracy_score(y_test, y_pred)

0.7496855345911949